# Building RAG Baseline

In [12]:
!pip install -q faiss-cpu rank-bm25 groq

In [13]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from rank_bm25 import BM25Okapi
import re
from groq import Groq

In [14]:
data_path = "/kaggle/input/rag-project/news_subset.csv"

df = pd.read_csv(data_path)
print(df.head())

       id                                              title  \
0   74444  وزیر دفاع دولت پیشبرد امور لبنان، سرپرستی وزار...   
1  181228              ۶۶۰ یتیم در استان سمنان حمایت می‌شوند   
2  162347  وال‌استریت‌ژورنال: ایران تولید تجهیزات سانتریف...   
3   44572  ۱۸ بقعه استان مرکزی میزبان طرح نوروزی و آرامش ...   
4   14794  شیره‌کش‌خانه‌های لاکچری در رژیم پهلوی!/ کار و ...   

                  short_link      service       subgroup  \
0        http://fna.ir/1d3ps    بین الملل       غرب آسیا   
1  http://mehrnews.com/xZdsH      استانها          سمنان   
2        http://fna.ir/4wug9    بین الملل  آمریکا، اروپا   
3       http://fna.ir/f354b5      استانها          مرکزی   
4       http://fna.ir/f2fm8j  آرشیو اخبار  دیگر رسانه ها   

                                            abstract  \
0  پس از استعفای وزیر امور خارجه دولت پیشبرد امور...   
1  سمنان- مدیرکل کمیته امداد استان سمنان گفت: ماه...   
2  وال‌استریت‌ژورنال: ایران تولید تجهیزات سانتریف...   
3  مدیرکل اوقاف و امور خیریه ا

## Chunking the dataset documents

In [15]:
def chunking(df):
    chunks = []
    for i in range(len(df)):
        text = df['title'].iloc[i] + '\n' + df['body'].iloc[i]
        
        if len(text)<=800:
            row = [0, i, text, df['subgroup'].iloc[i], df['published_datetime'].iloc[i]]
            chunks.append(row)
        else:
            # Calculating the number of chunks
            n_chunks = int(np.ceil((len(text) - 800) / 650)) + 1
            for c in range(n_chunks):
                row = [c, i, text[c*650:c*650+800], df['subgroup'].iloc[i], df['published_datetime'].iloc[i]]
                chunks.append(row)
    return pd.DataFrame(data=chunks, columns=['chunk_id', 'doc_id', 'text', 'category', 'date'])

chunks_df = chunking(df)
print(chunks_df)

       chunk_id  doc_id                                               text  \
0             0       0  وزیر دفاع دولت پیشبرد امور لبنان، سرپرستی وزار...   
1             1       0  زمان داعش آوردند و [بذر] آن ها را برای ما در د...   
2             0       1  ۶۶۰ یتیم در استان سمنان حمایت می‌شوند\nبه گزار...   
3             1       1   شود. ذوالفقاری با بیان اینکه در این طرح، افرا...   
4             0       2  وال‌استریت‌ژورنال: ایران تولید تجهیزات سانتریف...   
...         ...     ...                                                ...   
69857         2   19998  ارش ۱۰۰ میلیمتری در بالادست سدهای استان در ۲ ه...   
69858         3   19998  متر مکعب بر ثانیه رسیده است تا بتوانیم حجم سد ...   
69859         4   19998  تاهای واقع در مسیر رودخانه تخلیه شده است تا سی...   
69860         0   19999  ۱۰۷ مادر و همسر شهید  در خورموج تجلیل شدند\nبه...   
69861         1   19999   ام البنین (سلام الله علیها) هدیه گرفته اند. و...   

            category                    date  
0           غرب 

## Lexical Embedding by BM-25

In [16]:
def tokenize(text):
    return re.findall(r"\w+", text.lower())

tokenized_docs = [tokenize(c) for c in chunks_df['text']]
bm25 = BM25Okapi(tokenized_docs)

## Semantic Embedding by paraphrase-multilingual-MiniLM-L12-v2

In [17]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(chunks_df['text'], normalize_embeddings=True)

# Making FAISS Index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Retrieval Function

In [18]:
def retrieve(question, embd_model, top_k=5):
    if embd_model == 'lexical':
        bm25_scores = bm25.get_scores(tokenize(question))
        top_indices = np.argsort(bm25_scores)[::-1][:top_k]
        bm25_docs = [chunks_df['text'][i] for i in top_indices]
        doc_ids = [chunks_df['doc_id'][i] for i in top_indices]
        return bm25_docs, doc_ids

    if embd_model == 'semantic':
        q_embd = model.encode(question, normalize_embeddings=True)
        scores, idx = index.search(q_embd.reshape(1, -1), k=top_k)
        semantic_docs = [chunks_df['text'][i] for i in idx[0]]
        doc_ids = [chunks_df['doc_id'][i] for i in idx[0]]
        return semantic_docs, doc_ids

    # First retrieve 20 chunks by BM-25 then rerank using semantic model
    if embd_model == 'hybrid':
        bm25_scores = bm25.get_scores(tokenize(question))
        top_indices = np.argsort(bm25_scores)[::-1][:20]

        q_embd = model.encode(question, normalize_embeddings=True)
        scores = np.array([embeddings[i]@q_embd.T  for i in top_indices])
        idx = np.argsort(scores)[::-1][:top_k]
        reranked_idx = [top_indices[i] for i in idx]
        hybrid_docs = [chunks_df['text'][i] for i in reranked_idx]
        doc_ids = [chunks_df['doc_id'][i] for i in reranked_idx]
        return hybrid_docs, doc_ids

## Generate answer with LLM

In [19]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("groq")

client = Groq(api_key=API_KEY)

sys_prompt = """You are a helpful assistant. Your role is to answer user's questions, based on
the potentially relevant pieces of Persian news given to you as context.
If the given context is not relevant, say that you don't have enough information to answer.
Always answer in Persian. Try to answer as concisely as possible, without missing important details.
"""

def answer(query, embd_model):
    context_docs, ids = retrieve(query, embd_model)
    sep = '\n' + '-'*50 + '\n'
    context = sep.join(context_docs)
    
    prompt = f"""This is the context:
    {context}
    **********************
    Based on the context answer the user's question.
    Question: {query}
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ],
        model="llama-3.1-8b-instant",
        temperature=1,
        max_completion_tokens=1024
    )
    response = chat_completion.choices[0].message.content
    return response, ids, context_docs


In [20]:
query = "سرمربی تیم بارسلونا کیه؟"
result, doc_ids, _ = answer(query, 'hybrid')
print("Based on documents:", doc_ids)
print(result)

Based on documents: [np.int64(9716), np.int64(14036), np.int64(756), np.int64(10275), np.int64(16561)]
رونالد کومان سرمربی هدایت تیم بارسلونا را برعهده دارد.


In [21]:
df['body'][9716]

'به گزارش خبرگزاری فارس، امروز انتخابات بارسلونا برای ریاست بعدی باشگاه برگزار خواهد شد و ۱۱۰۲۹۰ عضو در این انتخابات شرکت خواهند کرد. از این تعداد ۲۰۶۶۳ عضو به صورت الکترونیکی رای خواهند داد که این سیستم برای اولین بار به خاطر ویروس کرونا اجرا خواهد شد. ریاست بعدی باشگاه از بین خوان لاپورتا، ویکتور فونت و تونی فریشا انتخاب خواهد شد و باید دید اعضای هواداری باشگاه بارسلونا چه کسی برای ادامه راه انتخاب خواهند کرد. در گزارش زیر به بررسی برنامه های کاندیداها خواهیم پرداخت اما ابتدا مروری بر عملکرد روسای باشگاه بارسلونا از سال ۲۰۰۳ تا ۲۰۲۰ داشته باشیم. *لاپورتا – راسل – بارتومیو (آغاز شکوه بارسلونا تا رسوایی بزرگ) از سال ۲۰۰۳ خوان لاپورتا مدیریت باشگاه را برعهده گرفت و توانست با آوردن فرانک رایکارد، رونالدینیو و اهتمام به مدرسه فوتبال بارسلونا، این تیم را احیا کند و به قله فوتبال برساند. در سال ۲۰۰۸ لاپورتا، سرمربی وقت بارسلونای ب یعنی پپ گواردیولا را جانشین رایکارد کرد تا سلطه «تیکی تاکا» ، گواردیولا و لیونل مسی آغاز شود و دوره جدیدی بارسلونا در تاریخ بسازد. لاپورتا با ۱۲ جام در سال ۲۰۱۰ ج

# Evaluation and Report

In [22]:
temp_path = "/kaggle/input/qa-template/questions_template.jsonl"
qa_template = pd.read_json(temp_path, lines=True)
print(qa_template)

     id                                           question  \
0    q1  در کدام شهر و ایالت آمریکا ۸ عضو یک خانواده به...   
1    q2  چه کسی در اجلاس بین‌المللی همکاری‌های علمی و ا...   
2    q3  شاخص کل بورس اوراق بهادار تهران در پایان معامل...   
3    q4  چه تعداد مجوز بهداشتی و قرنطینه‌ای حمل دام و ف...   
4    q5  بسیج جامعه پزشکی برای جبران کمبود پزشک در مناط...   
5    q6  تعداد بیماران کرونایی بستری در مراکز درمانی گل...   
6    q7  شهاب عزیزی خادم در مورد اجرای طرح استعدادیابی ...   
7    q8           موضوع مجموعه تلویزیونی «شبکه کوچک» چیست؟   
8    q9        کدام مجری پربیننده شبکه فاکس نیوز اخراج شد؟   
9   q10           ورودی و خروجی آب سد زاینده‌رود چقدر است؟   
10  q11  در طرح نوروزی پلیس امنیت اقتصادی چند نفر متهم ...   
11  q12  تیم بسکتبال شیمیدر قم در مرحله مقدماتی چند برد...   
12  q13  اشیای عتیقه تقلبی کشف شده در مراغه از چه موادی...   
13  q14                         هدف طرح «بنای ایران» چیست؟   
14  q15  کدام باشگاه فرانسوی به سردار آزمون پیشنهاد داد...   

       

In [23]:
# Convert the JSON lines into a list of dicts
def get_qas():
    qa_list = []
    qa_len = len(qa_template)
    for i in range(qa_len):
        qa_list.append(dict(qa_template.iloc[i]))
    return qa_list

get_qas()

[{'id': 'q1',
  'question': 'در کدام شهر و ایالت آمریکا ۸ عضو یک خانواده به دلیل تیراندازی کشته شدند؟',
  'reference_answer': 'این حادثه در شهر انوک در ایالت یوتای آمریکا رخ داد.',
  'doc_ids': [15956]},
 {'id': 'q2',
  'question': 'چه کسی در اجلاس بین\u200cالمللی همکاری\u200cهای علمی و اقتصادی ایران و کشورهای غرب آفریقا پیشنهاد تاسیس بانک مشترک را داد؟',
  'reference_answer': 'محسن رضایی، معاون اقتصادی رئیس جمهوری، پیشنهاد تاسیس بانک مشترک ایران و آفریقا را داد.',
  'doc_ids': [12585]},
 {'id': 'q3',
  'question': 'شاخص کل بورس اوراق بهادار تهران در پایان معاملات چقدر کاهش یافت؟',
  'reference_answer': 'شاخص کل بورس اوراق بهادار تهران با کاهش ۱۸ هزار و ۷۴۶ واحد به رقم یک میلیون و ۱۵۴ هزار و ۷۶۷ واحد رسید.',
  'doc_ids': [8144]},
 {'id': 'q4',
  'question': 'چه تعداد مجوز بهداشتی و قرنطینه\u200cای حمل دام و فرآورده\u200cهای خام دامی طی سال گذشته صادر شده است؟',
  'reference_answer': 'طی سال گذشته ۸۰۰۰ مورد مجوز بهداشتی و قرنطینه\u200cای حمل انواع دام و فرآورده\u200cهای خام دامی صادر شد

In [24]:
# Evaluating the RAG system and saving the results
import time

qa_list = get_qas()
methods = ['lexical', 'semantic', 'hybrid']
eval_scores = {"lexical": 0, "semantic": 0, "hybrid": 0}
# List of dicts to save the retrieved chunks for each question and other data
eval_data = []

for qa in qa_list:
    print('-'*50)
    print( 'Question: ' + f'{qa["question"]}')
    print( 'True Answer: ' + f'{qa["reference_answer"]}')
    print('-'*50)
    
    for method in methods:
        result, doc_ids, retrieved_chunks = answer(qa['question'], method)
        print('*'*30 + f" ({method.upper()}) " + '*'*30)
        # Check if the true reference document is retrieved
        found_ref = False
        if qa['doc_ids'][0] in doc_ids:
            found_ref = True
            eval_scores[method] += 1
            print("True reference retrieved!")

        else:
            print("Failed to retrieve the true reference!")
            
        print('LLM Answer:')
        print(result)
        data = {'id': qa['id'], 'question': qa['question'], 'ref_answer': qa['reference_answer'],
                'method': method, 'llm_answer': result, 'retrieved_chunks': retrieved_chunks,
                'found_reference': found_ref
               }
        eval_data.append(data)
        # Avoiding the rate limit
        time.sleep(2)


--------------------------------------------------
Question: در کدام شهر و ایالت آمریکا ۸ عضو یک خانواده به دلیل تیراندازی کشته شدند؟
True Answer: این حادثه در شهر انوک در ایالت یوتای آمریکا رخ داد.
--------------------------------------------------
****************************** (LEXICAL) ******************************
True reference retrieved!
LLM Answer:
با توجه به اخبار منتشر شده، ۸ عضو یک خانواده به دلیل تیراندازی کشته شدن در شهر انوک، ایالت یوتای آمریکا رخ داده است.
****************************** (SEMANTIC) ******************************
True reference retrieved!
LLM Answer:
بر اساس گزارش‌ها، در یک شهر نامنوشته در ایالت یوتای آمریکا، ۸ عضو یک خانواده به دلیل تیراندازی کشته شدند، اما نام شهر مشخص نیست.
****************************** (HYBRID) ******************************
True reference retrieved!
LLM Answer:
بر اساس اخبار منتشر شده، این حادثه در شهر "انوک"، ایالت "یوتا" در آمریکا رخ داد.
--------------------------------------------------
Question: چه کسی در اجلاس بین‌المللی همکار

In [25]:
eval_df = pd.DataFrame(eval_data)
print(eval_df)

     id                                           question  \
0    q1  در کدام شهر و ایالت آمریکا ۸ عضو یک خانواده به...   
1    q1  در کدام شهر و ایالت آمریکا ۸ عضو یک خانواده به...   
2    q1  در کدام شهر و ایالت آمریکا ۸ عضو یک خانواده به...   
3    q2  چه کسی در اجلاس بین‌المللی همکاری‌های علمی و ا...   
4    q2  چه کسی در اجلاس بین‌المللی همکاری‌های علمی و ا...   
5    q2  چه کسی در اجلاس بین‌المللی همکاری‌های علمی و ا...   
6    q3  شاخص کل بورس اوراق بهادار تهران در پایان معامل...   
7    q3  شاخص کل بورس اوراق بهادار تهران در پایان معامل...   
8    q3  شاخص کل بورس اوراق بهادار تهران در پایان معامل...   
9    q4  چه تعداد مجوز بهداشتی و قرنطینه‌ای حمل دام و ف...   
10   q4  چه تعداد مجوز بهداشتی و قرنطینه‌ای حمل دام و ف...   
11   q4  چه تعداد مجوز بهداشتی و قرنطینه‌ای حمل دام و ف...   
12   q5  بسیج جامعه پزشکی برای جبران کمبود پزشک در مناط...   
13   q5  بسیج جامعه پزشکی برای جبران کمبود پزشک در مناط...   
14   q5  بسیج جامعه پزشکی برای جبران کمبود پزشک در مناط...   
15   q6 

In [26]:
eval_df.to_json('test_results.json', indent=2)

In [27]:
eval_scores

{'lexical': 15, 'semantic': 10, 'hybrid': 13}

<div dir='rtl'>
<h2> خلاصه ی نتایج</h2>

- lexical: در ۱۵ مورد از ۱۵ سوال، reference واقعی retrieve شده (امتیاز ۱۵).


- semantic: در ۱۰ مورد موفق (امتیاز ۱۰).


- hybrid: در ۱۳ مورد موفق (امتیاز ۱۳).

<h3>تحلیل</h3>

- روش lexical در retrieval دقیق‌تر عمل کرده، احتمالاً به دلیل تمرکز بر کلمات کلیدی فارسی.


- روش semantic ممکن است به دلیل پیچیدگی‌های زبانی فارسی (مانند هم‌معنی‌ها) ضعیف‌تر باشد.


- hybrid تعادلی بین دو روش ایجاد کرده و عملکرد بهتری نسبت به semantic خالص دارد.


- در برخی سوالات (مانند q13 و q14)، همه روش‌ها موفق بوده‌اند، اما در مواردی مانند q15 (باشگاه فرانسوی برای سردار آزمون)، semantic و hybrid شکست خورده‌اند.


<h3>نقاط قوت:</h3>

پشتیبانی از زبان فارسی، ترکیب روش‌های retrieval، استفاده از مدل‌های پیشرفته مانند SentenceTransformer و Llama.

<h3>نقاط ضعف:</h3>

وابستگی به حجم chunk ها، ممکن است در سوالات پیچیده‌تر شکست بخورد، و ارزیابی محدود به ۱۵ سوال. همچنین خبر ها به روز نیستند.


</div>
